# Ideal gas equation of state using grand canonical ensemble transition-matrix Monte Carlo

In this example, the ideal gas equation of state is obtained as a test of the flat histogram method.

In [1]:
import feasst as fst

monte_carlo = fst.MonteCarlo()
monte_carlo.add(fst.Configuration(fst.MakeDomain(fst.args({"cubic_box_length": "8"})),
                                  fst.args({"particle_type": fst.install_dir()+'/forcefield/data.atom'})))
monte_carlo.add(fst.Potential(fst.MakeDontVisitModel()))
criteria = fst.MakeFlatHistogram(
    fst.MakeMacrostateNumParticles(fst.Histogram(fst.args({"width": "1", "min": "0", "max": "50"}))),
    fst.MakeTransitionMatrix(fst.args({"min_sweeps": "100"})),
    fst.args({"beta": str(1./1.2), "chemical_potential": "-3"}))
monte_carlo.set(criteria)
monte_carlo.add(fst.MakeTrialTransfer(fst.args({"particle_type": "0"})))
monte_carlo.add(fst.MakeCriteriaUpdater(fst.args({"steps_per": str(1e5)})))
monte_carlo.add(fst.MakeCriteriaWriter(fst.args({"steps_per": str(1e5), "file_name": "id_fh.txt"})))

In [2]:
%%time
monte_carlo.run_until_complete()

CPU times: user 11.7 s, sys: 11.1 ms, total: 11.7 s
Wall time: 11.7 s


Check the ideal gas relationship, $\beta P = \rho$

In [3]:
import numpy as np

volume = monte_carlo.configuration().domain().volume()
print('rho beta*P difference')
for delta_conjugate in np.arange(-5, 5, 0.1):
        lnpirw = criteria.reweight(delta_conjugate)
        if lnpirw.value(lnpirw.size()-1) < -6:
            rho = criteria.average_macrostate(lnpirw)/volume
            pressure = criteria.pressure(lnpirw, volume)
            print(rho, criteria.beta()*pressure, rho - criteria.beta()*pressure)

rho beta*P difference
0.0005525928807084006 0.0005524447899680177 1.48090740382926e-07
0.0006107356478535072 0.0006105627467278755 1.7290112563168522e-07
0.0006749966045572873 0.0006747957913424284 2.008132148589555e-07
0.0007460192305781728 0.0007457873816922129 2.3184888595984866e-07
0.000824514515492435 0.0008242486433600914 2.658721323436095e-07
0.0009112679896123723 0.0009109654664323808 3.025231799915016e-07
0.0010071474763284941 0.0010068063408849772 3.4113544351691873e-07
0.0011131116386788394 0.0011127310062791287 3.806323997106349e-07
0.0012302194001342702 0.0012297999991272715 4.1940100699864105e-07
0.001359640327127634 0.0013591851896634023 4.551374642317377e-07
0.001502666068424659 0.0015021814088851811 4.846595394777508e-07
0.0016607229535468998 0.001660219276606663 5.036769402367268e-07
0.0018353858585231431 0.0018348793517959604 5.065067271827275e-07
0.0020283934518215865 0.002027907737541538 4.857142800484583e-07
0.00224166493667871 0.002241233284449657 4.3165222905284

Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!